In [1]:
import numpy as np
import pandas as pd
from copy import copy
from pathlib import Path
from itertools import chain
from tqdm.notebook import tqdm

import data_tools.graphs as gt
from hetnet_ml.extractor import MatrixFormattedGraph, piecewise_extraction

/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import utils

In [3]:
param_file = 'CtP_param.json'
param_file = Path('../0_data/manual').resolve().joinpath(param_file)

In [4]:
tgt_node, tgt_edge = utils.load_targets(param_file)

In [5]:
nodes, edges = utils.load_network(param_file)

In [6]:
# Hyperparameter tuning is param #2 (network is 1, model trainig is 3)
out_dir = utils.read_data_param(param_file)[1]

In [7]:
nodes.head()

,id,name,label,xrefs,source,synonyms,alt_ids,subsets
0,UBERON:0000002,cervix,AnatomicalEntity,MESH:D002584|UBERON:0000002,NaN,NaN,NaN,NaN
1,UBERON:0000004,human nose,AnatomicalEntity,MESH:D009666|UBERON:0000004,NaN,NaN,NaN,NaN
2,UBERON:0000006,islet of Langerhans,AnatomicalEntity,MESH:D007515|UBERON:0000006,NaN,NaN,NaN,NaN
3,UBERON:0000007,pituitary gland,AnatomicalEntity,MESH:D010902|UBERON:0000007,NaN,NaN,NaN,NaN
4,UBERON:0000010,peripheral nervous system,AnatomicalEntity,MESH:D017933|UBERON:0000010,NaN,NaN,NaN,NaN


In [8]:
# load the params from the original model to get the W
# Won't be using the old features...
params, old_feats = utils.load_hyperparameters()

In [9]:
mg = MatrixFormattedGraph(nodes, edges, 'ChemicalSubstance', tgt_node, 
                          max_length=4, w=params['w'], n_jobs=30)

Processing node and edge data...
Initializing metagraph...
Generating adjacency matrices...


100%|██████████| 68/68 [00:55<00:00,  1.24it/s]



Determining degrees for each node and metaedge


100%|██████████| 68/68 [00:26<00:00,  2.55it/s]



Weighting matrices by degree with dampening factor 0.5665616931869023...


100%|██████████| 68/68 [00:00<00:00, 117.82it/s]


# Data Transformer

We use an inverse hyperbolic sine transformation to transform the features.

The transormation is thus:

$$sinh^{-1} \left( \frac{X_{mp}}{\sigma_{mp}} \right)$$

where $X_{mp}$ is the column in the dwpc feature matrix $X$ coreesponding to metapath $mp$ and $\sigma_{mp}$ is the mean of the column.

In [10]:
from data_tools.ml import MeanScaledArcsinhTransformer, get_model_coefs

# Homegrown Feature Selector

Runs 6 analysis on the traning data to select features.

1. Correlation to the output
2. Chi_squared test
3. Recurssive Feature Elimantion on a Ridge Regressor
4. Embedded Feature Selection from a Lasso Regressor
5. Embedded Feature Selection from a Randomn Forest Classifier
6. Embedded Feature Selection from a Gradient Boosting Classifier

Each analysis will select `num_feats` best features. The selected features will then by chosen via a voting method with `min_selections` out of the 6 elements required to for a feature to be kept. 


We have also added an option for `always_keep`:  This allows for domain expertise to be factored into the feature selection process.  In our case, we know some metapaths are specifically mechanistic, so we want to include those wherever possible

In [11]:
from scipy.sparse import issparse, csc_matrix, csr_matrix

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel

from xgboost import XGBClassifier

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [12]:
def cor_selector(X, y, feature_names, num_feats):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in range(X.shape[1]):
        if issparse(X):
            x = X[:, i].A.reshape(len(y))
        else:
            x = X[:, i]
        cor = np.corrcoef(x, y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = np.array(feature_names)[np.argsort(np.abs(cor_list))[-num_feats:].tolist()].tolist()    
    # feature selection? 0 for not select, 1 for select
    return [True if i in cor_feature else False for i in feature_names]

def chi2_selector(X, y, num_feats):
    this_selector = SelectKBest(chi2, k=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()

def rfe_selector(X, y, num_feats, random_state=None):
    this_selector = RFE(estimator=LogisticRegression(C=.1, solver='liblinear', random_state=random_state), 
                        n_features_to_select=num_feats, step=.2, verbose=5)
    this_selector.fit(X, y)
    return this_selector.get_support()

def embeded_lr_selector(X, y, num_feats, random_state=None):
    this_selector = SelectFromModel(LogisticRegression(penalty="l1", solver='liblinear', random_state=random_state), 
                                    max_features=num_feats)
    this_selector.fit(X, y)

    return this_selector.get_support()

def embeded_rf_selector(X, y, num_feats, n_jobs, random_state=None):
    rfc = RandomForestClassifier(n_estimators=100, max_depth=50, n_jobs=n_jobs, random_state=random_state)
    this_selector = SelectFromModel(rfc, max_features=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()

def embeded_xgb_selector(X, y, num_feats, n_jobs=1, random_state=None):
    # XGBoost takes 0 as default random state
    if random_state is None:
        random_state = 0
    # Paramaters optimized for speed, rather than accuracy (as we have 5 other estimators also providing votes)
    xgbc = XGBClassifier(max_depth=5, n_estimators=200, learning_rate=.16, min_child_weight=1, colsample_bytree=.8,
                         n_jobs=n_jobs, random_state=random_state)
    this_selector = SelectFromModel(xgbc, max_features=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()


class FeatureSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_features=100, min_selections=4, n_jobs=1, feature_names=None, always_keep=None,
                 random_state=None):
        self.num_features = num_features
        self.min_selections = min_selections
        self.n_jobs = n_jobs
        self.feature_names = feature_names
        self.always_keep = always_keep
        self.random_state = random_state
        
    def fit(self, X, y=None):
                
        X_norm = MaxAbsScaler().fit_transform(X)
        if issparse(X):
            if type(X) != csc_matrix:
                X = X.tocsc()
            X_norm = X_norm.tocsc()
        
        print('Running Cor')
        cor_support = cor_selector(X, y, self.feature_names, self.num_features)
        print('Running Chi2')
        chi_support = chi2_selector(X_norm, y, self.num_features)
        print('Running RFE')
        rfe_support = rfe_selector(X_norm, y, self.num_features, self.random_state)
        print('Running LR')
        embeded_lr_support = embeded_lr_selector(X_norm, y, self.num_features, self.random_state)
        print('Running RF')
        embeded_rf_support = embeded_rf_selector(X, y, self.num_features, 
                                                 n_jobs=self.n_jobs, random_state=self.random_state)
        print('Running XG')
        embeded_xgb_support = embeded_xgb_selector(X, y, self.num_features, 
                                                   n_jobs=self.n_jobs, random_state=self.random_state)
        
        feature_selection_df = pd.DataFrame({'feature':self.feature_names, 'pearson':cor_support, 'chi_2':chi_support, 
                                             'rfe':rfe_support, 'logistics':embeded_lr_support,
                                             'random_forest':embeded_rf_support, 'xgboost':embeded_xgb_support})  

        feature_selection_df['total'] = np.sum(feature_selection_df, axis=1)
        self.feature_selection_df_ = feature_selection_df
        
        keep_features = feature_selection_df.query('total >= {}'.format(self.min_selections))['feature'].tolist()
        
        # Keep the features that we always want (e.g. domain expertise)
        if self.always_keep is not None:
            keep_features.extend(self.always_keep)
        
        self.keep_features_ = [f for f in self.feature_names if f in keep_features]
        
        return self
    
    def transform(self, X, y=None):
        
        if issparse(X) and type(X) != csc_matrix:
            X = X.tocsc()
        return X[:, [i for i, f in enumerate(self.feature_names) if f in self.keep_features_]]

In [13]:
from data_tools import graphs as gt
from collections import Counter

In [14]:
edge_info = pd.read_csv('../metapaths/0_data/manual/edge_semtypes.csv')

In [15]:
dir_map = edge_info.set_index('fwd_edge')['directed'].to_dict()
rev_map = edge_info.set_index('fwd_edge')['rev_edge'].to_dict()
direction = edge_info.set_index('fwd_edge')['rel_dir'].to_dict()

In [17]:
similarity_paths_check = []

for mp in tqdm(mg.metapaths):
    if len(mp) < 2:
        similarity_paths_check.append(True)
        continue
    this_path = []

    this_path.append(gt.is_similarity(mp, ['ChemicalSubstance'], dir_map, check_dir='both'))
    this_path.append(gt.is_similarity(mp, ['MacromolecularMachine'], dir_map, check_dir='both', max_repeats=3))
    this_path.append(gt.is_similarity(mp, ['Disease', 'PhenotypicFeature'], dir_map, max_repeats=1))

    # Don't want CtXaYzP edges.. Ct (and Cm) edges should be banned...
    # Though not strictly similarity edges, they have heavy implications for Disease Phenotype similarity
    bl_edge = ['Ct', 'Cm', 'Cdg', 'Cpl', 'Cpv']
    for bl in bl_edge:
        if mp.abbrev.startswith(bl) or mp.abbrev.endswith('aw'+bl+'P'):
            this_path.append(True)
    
    similarity_paths_check.append(sum(this_path) > 0)

In [18]:
non_sim_mps = [mp for mp, sim in zip(mg.metapaths, similarity_paths_check) if not sim]
len(non_sim_mps)

3315

In [19]:
combo = gt.combine_nodes_and_edges(nodes, edges)
combo.query('start_label == "ChemicalSubstance" and end_label == @tgt_node')['type'].value_counts()

marker_or_mechanism_CmP    4909
treats_CtP                 3478
palliates_CplP              221
Name: type, dtype: int64

In [20]:
keep_comps = set(edges.query('type == @tgt_edge')['start_id'].tolist())
keep_bp = set(edges.query('type == @tgt_edge')['end_id'].tolist())

In [32]:
train_frac = 0.75
rs = 20201116

# Get compounds that have our target edge
tgt_comps = set(edges.query('type == @tgt_edge')['start_id'])
# Sample the comps without our target edge, then subsample
keep_comps = set(nodes.query('id not in @tgt_comps and label == "ChemicalSubstance"')
                      .sample(frac=train_frac*.01, random_state=rs)['id'])
# Then subsample the positives
keep_comps = keep_comps | set(nodes.query('id in @tgt_comps')
                                   .sample(frac=train_frac, random_state=rs+1)['id'])

# Get our target type 
tgt_bps = set(edges.query('type == @tgt_edge')['end_id'])
# Sample the negatives and subsample cv
keep_bps = set(nodes.query('label == @tgt_node and id not in @tgt_bps')
                    .sample(frac=train_frac*.01, random_state=rs+2)['id'])
# Take the diseases Treated by these compounds
keep_bps = keep_bps | set(edges.query('type == @tgt_edge and start_id in @keep_comps')['end_id'])

In [33]:
len(keep_comps), len(keep_bp)

(1801, 328)

In [34]:
len(keep_comps) * len(keep_bp)

590728

In [35]:
print('{:,} Positive training examples in subset'.format(len(edges.query('start_id in @keep_comps and type == @tgt_edge'))))

2,670 Positive training examples in subset


# Prepare a tmp output location

In [36]:
tmp_dir = out_dir.parent.joinpath('tmp')
tmp_dir.mkdir(parents=True, exist_ok=True)

# Extract the features one time

Use the pair counts to sort metapaths extaction as a niave load balancer

In [37]:
to_xtract = [str(mp) for mp in non_sim_mps]

In [38]:
%%time
## Extract the metapaths to do some prep work
(pairs, feats), test_dwpc = piecewise_extraction(function=mg.extract_dwpc, 
                                 to_split='metapaths', block_size=200,
                                 axis=1,
                                 metapaths=to_xtract, 
                                 start_nodes=list(keep_comps), 
                                 end_nodes=list(keep_bps),
                                 return_sparse=True,
                                 sparse_df=False,
                                 n_jobs=30)

100%|██████████| 17/17 [03:06<00:00, 10.99s/it]


CPU times: user 1min 8s, sys: 1min 4s, total: 2min 13s
Wall time: 3min 7s


# Training Targets for feature selection¶

In [39]:
pos_tups = edges.query('type == @tgt_edge')[['start_id', 'end_id']].apply(tuple, axis=1).tolist()
pos_tups[:2]

[('CHEBI:100147', 'HP:0002740'), ('CHEBI:100241', 'HP:0000010')]

In [40]:
pos_tups = set(pos_tups)

In [41]:
y = []
for row in tqdm(pairs.itertuples(), total=len(pairs)):
    if set([(row.chemicalsubstance_id, getattr(row, tgt_node.lower()+'_id'))]) & pos_tups:
        y.append(1)
    else:
        y.append(0)

y = np.array(y)
        
print(len(pairs), len(y), sum(y))


619544 619544 2670


In [42]:
pairs['status'] = y

# Subsample the potnetial training examples

Our class is so imbalanced, to get a sizeable number of positive training examples, we end up with many orders of magnitude more negative examples.  Many of those examples will have no connections from the compound to the disease of interest, this providing a zero row in the matrix. We will not waste time training on those values, and instead focus on the ones that distinguish the positive from the negative training examples.

As this is just hyperparameter tunings, to speed things up, we will also limit the positive examples to a small portion of the negative examples, 100x (2 orders of magnitude) larger than the number of postitives. 

In [43]:
# Get the rows that are Non-zero
nz_index = pairs[test_dwpc.getnnz(1)>0].index

# have the number of postivies to get 100x this for the negatives.
n_pos = pairs['status'].sum()

# Sample the nonzero negative examples at a rate of 100x the positive samples
neg_index = pairs.loc[nz_index].query('status == 0').sample(n=100*n_pos, random_state=rs+10).sort_index().index

# and of course take the training postivies
pos_index = pairs.query('status == 1').index

# Union the two
train_index = pos_index.union(neg_index)

In [44]:
feats = np.array(feats)
nz_feats = feats[test_dwpc.getnnz(0)>0]
feat_index = test_dwpc.getnnz(0)>0

In [45]:
# Get our compounds as ndarrays for easy indexing with sklearns StratifiedKFold
keep_comps = np.array(list(keep_comps))
# Need to know how to properly stratify the split
is_treat_comp = np.array([1 if c in tgt_comps else 0 for c in keep_comps])

## Select the features

This is a time consuming and costly step. We will do once with the initial DWPC for parameter tuning. We will perform again at the end with the selected parameters.

In [46]:
%%time
msat = MeanScaledArcsinhTransformer()
trans_dwpc = msat.fit_transform(test_dwpc[train_index][:, feat_index])

CPU times: user 10.2 s, sys: 2.28 s, total: 12.5 s
Wall time: 12.5 s


In [47]:
%%time

if tmp_dir.joinpath('test_feats.txt').exists():
    keep_features = pd.read_csv(tmp_dir.joinpath('test_feats.txt'))['0'].tolist()
else:
    fsel = FeatureSelector(num_features=500, min_selections=4, n_jobs=30, 
                           feature_names=nz_feats.tolist(), random_state=rs+5)
    sel_dwpc = fsel.fit_transform(trans_dwpc, y[train_index])
    keep_features = fsel.keep_features_
    pd.Series(keep_features).to_csv(tmp_dir.joinpath('test_feats.txt'), index=False)
    fsel.feature_selection_df_.to_csv(tmp_dir.joinpath('test_fs_df.csv'), index=False)

Running Cor


/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Running Chi2
Running RFE
Fitting estimator with 2360 features.
Fitting estimator with 1888 features.
Fitting estimator with 1416 features.
Fitting estimator with 944 features.
Running LR
Running RF
Running XG
CPU times: user 1h 17min 33s, sys: 20.2 s, total: 1h 17min 53s
Wall time: 5min


## Prep for Hyperparam optimization

In [48]:
def neg_log_scal(val):
    """Scale values logrithmicly. Larger input yields smaller result"""
    return (1-np.log1p(val)) 

def pos_log_scal(val):
    """Scale values logrithmicly. Larger input yields larger result"""
    return np.log1p(val)


def auroc_mean_loss(auroc_mean, strength=2, scal_factor=.5):
    mean_roc_shift =  (np.abs(auroc_mean - 0.5)) / scal_factor
    baseline_roc = neg_log_scal(1)
    return (neg_log_scal(mean_roc_shift) - baseline_roc) * strength

def avg_pre_mean_loss(avg_pre_mean, strength=2.5, scal_factor=.15):
    avg_pre_mean_scal = avg_pre_mean / scal_factor
    baseline_prc = neg_log_scal(1)
    return (neg_log_scal(avg_pre_mean_scal) - baseline_prc) * strength

def auroc_std_loss(auroc_std, strength=2):
    return pos_log_scal(auroc_std) * strength

def avg_pre_std_loss(avg_pre_std, strength=2):
    return pos_log_scal(avg_pre_std) * strength

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std):
    mean_roc_loss = auroc_mean_loss(auroc_mean)
    mean_pre_loss = avg_pre_mean_loss(avg_pre_mean)
    std_roc_loss = auroc_std_loss(auroc_std)
    std_pre_loss = avg_pre_std_loss(avg_pre_std)
    
    return sigmoid(mean_roc_loss + mean_pre_loss + std_roc_loss + std_pre_loss)-1
    
    

In [49]:
import pickle
from time import time

def hyperopt(param_space, y, num_eval):

    def objective_function(params):
        dwpc_params = {k.split('__')[1]: v for k, v in params.items() if k.split('__')[0] == 'dwpc'}
        enet_params = {k.split('__')[1]: v for k, v in params.items() if k.split('__')[0] == 'enet'}

        # Set up the post feature extraction pipeline
        post_extraction_pipeline = Pipeline(
            [('e_net', LogisticRegression(penalty='elasticnet', solver='saga', max_iter=100, **enet_params,
                                          random_state=rs+6))], verbose=True)

        cv = cross_validate(post_extraction_pipeline, sel_dwpc, y, cv=5, 
                            scoring=['average_precision', 'roc_auc'], return_estimator=True)

        # Write out scores for each run
        with open(tmp_dir.joinpath('scores_C_{0:1.5f}_l1_{1:1.4f}.txt'.format(
                                   enet_params['C'], enet_params['l1_ratio'])), 'w') as f_out:

            f_out.write(', '.join([str(s) for s in cv['test_average_precision']]))
            f_out.write('\n')
            f_out.write(', '.join([str(s) for s in cv['test_roc_auc']]))
            f_out.write('\n')

        auroc_mean = cv['test_roc_auc'].mean()
        avg_pre_mean = cv['test_average_precision'].mean()
        auroc_std = cv['test_roc_auc'].std()
        avg_pre_std = cv['test_average_precision'].std()
        
        score = calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std)
        
        print('Mean AUROC: {:1.4f}'.format(auroc_mean))
        print('Mean Avg Pre: {:1.4f}'.format(avg_pre_mean))
        print('STD AUROC: {:1.4f}'.format(auroc_std))
        print('STD Avg Pre: {:1.4f}'.format(avg_pre_std))
        print('Loss: {:1.4f}'.format(score))
        
        return {'loss': score, 'status': STATUS_OK}

    start = time()
   
    
    trials = Trials()
    best_param = fmin(objective_function, 
                      param_space, 
                      algo=tpe.suggest, 
                      max_evals=num_eval, 
                      trials=trials,
                      rstate= np.random.RandomState(1))
    
    print(time() - start)
    return trials, best_param
    

In [50]:
# Previous best before adding new keep_features: 
# '1l_ratio': 0.10455936193818496, 'C': 0.000556880900960339, 'w': 0.2640929485381926
param_hyperopt = {
    'enet__C': hp.loguniform('C', np.log(0.001), np.log(1)),
    'enet__l1_ratio': hp.uniform('l1_ratio', .01, .99),
}

In [51]:
trials, best_param = hyperopt(param_hyperopt, y[train_index], 50)

[Pipeline] ............. (step 1 of 1) Processing e_net, total=  52.6s
  0%|          | 0/50 [00:53<?, ?trial/s, best loss=?]

/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



[Pipeline] ............. (step 1 of 1) Processing e_net, total=  52.2s
  0%|          | 0/50 [01:46<?, ?trial/s, best loss=?]

/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



[Pipeline] ............. (step 1 of 1) Processing e_net, total=  51.0s
  0%|          | 0/50 [02:37<?, ?trial/s, best loss=?]

...

/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



[Pipeline] ............. (step 1 of 1) Processing e_net, total=  49.4s                
 98%|█████████▊| 49/50 [3:03:38<03:30, 210.18s/trial, best loss: -0.17555220166267327]

/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



[Pipeline] ............. (step 1 of 1) Processing e_net, total=  46.0s                
Mean AUROC: 0.6642                                                                    
Mean Avg Pre: 0.0894                                                                  
STD AUROC: 0.0486                                                                     
STD Avg Pre: 0.0536                                                                   
Loss: -0.1705                                                                         
100%|██████████| 50/50 [3:04:24<00:00, 221.29s/trial, best loss: -0.17555220166267327]
11064.468649625778


/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



In [52]:
best_param

{'C': 0.006334718111182533, 'l1_ratio': 0.029991113106311307}

In [53]:
trials.best_trial

{'state': 2,
 'tid': 20,
 'spec': None,
 'result': {'loss': -0.17555220166267327, 'status': 'ok'},
 'misc': {'tid': 20,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'C': [20], 'l1_ratio': [20]},
  'vals': {'C': [0.006334718111182533], 'l1_ratio': [0.029991113106311307]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2020, 11, 18, 7, 29, 36, 103000),
 'refresh_time': datetime.datetime(2020, 11, 18, 7, 33, 51, 467000)}

In [54]:
best_param['w'] = mg.w

In [55]:
utils.save_hyperparam(best_param, fsel.keep_features_, out_dir)